# Scientific Python tutorial for the STAR collaboration

This notebook shows you how to do physics analysis in Python using a STAR PicoDST file. I presented it on Zoom on September 13, 2021.

You can read it in GitHub ([jpivarski-talks/2021-09-13-star-uproot-awkward-tutorial](https://github.com/jpivarski-talks/2021-09-13-star-uproot-awkward-tutorial)) or run it [interactively on Binder](FIXME).

I recommend the latter, as that would let you see the outputs of the code cells and make modifications to explore "what if?" scenarios. Binder loads all dependencies on its server; you don't have to install anything.

If you want to run it on your own computer, see the `requirements.txt` file for Python packages to install. The requirements specifies exact versions that are known to work with this notebook, but if you plan to use these libraries for other work, you'll probably want the latest versions of each.

This notebook is broken up into sections that lead toward a project in which you will be translating a C++ analysis into array-oriented Python.

## 1. Python: interactively building up an analysis

## 2. NumPy: thinking one array at a time

## 3. Uproot: array-oriented ROOT I/O

## 4. Awkward Array: complex data in arrays

## 5. Vector, hist, fastjet: the array-oriented ecosystem

## 6. Z peak: translating a C++ analysis into array-oriented Python

In this section, you will plot the mass of e⁺e⁻ pairs from the PicoDST file, using [star-picodst-W-reference-for-jim/macros/PicoDstAnalyzer.C](star-picodst-W-reference-for-jim/macros/PicoDstAnalyzer.C) as a guide.

### Reading the data

All data come from the PicoDST file, so let's load it again.

In [59]:
import uproot
import awkward as ak
import numpy as np

In [3]:
picodst = uproot.open("https://pivarski-princeton.s3.amazonaws.com/pythia_ppZee_run17emb.picoDst.root:PicoDst")
picodst

<TTree 'PicoDst' (18 branches) at 0x7f651848f250>

In [star-picodst-W-reference-for-jim/macros/PicoDstAnalyzer.C](star-picodst-W-reference-for-jim/macros/PicoDstAnalyzer.C), the chain of commands that leads to getting a track is:

```c++
// get a reader and initialize it
const Char_t *inFile = "pythia_ppZee_run17emb.picoDst.root:PicoDst";
StPicoDstReader* picoReader = new StPicoDstReader(inFile);
picoReader->Init();

// in the loop over events (iEvent), load an event
Bool_t readEvent = picoReader->readPicoEvent(iEvent);
StPicoDst *dst = picoReader->picoDst();

// in the loop over tracks (iTrk), load a track
StPicoTrack *picoTrack = dst->track(iTrk);
```

The most relevant quantity for calculating invariant mass is the track's momentum:

```c++
picoTrack->pMom()
```

As a first step, we need to get track momenta. The [star-picodst-W-reference-for-jim/StPicoTrack.h](star-picodst-W-reference-for-jim/StPicoTrack.h) defines `pMom` as

```c++
TVector3 pMom() const           { return TVector3(mPMomentumX, mPMomentumY, mPMomentumZ); }
```

**Exercise 1:** Search through `picodst` and find _the full names_ of these branches. Then read all three into _arrays_ named `px`, `py`, and `pz`.

For reference, here is the [uproot.TTree](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html) documentation.

Depending on your network connection, those arrays could take 30 seconds to load. Be sure you have an expression that works on a small subsample before starting the big download.

In [ ]:
picodst. ???

<details>
    <summary><b>Answer</b> (no peeking!)</summary>

Search around with `picodst.keys`, gradually narrowing its output with wildcards or regular expressions like either of the following:

```python
picodst.keys(filter_name="*mPMomentum[XYZ]")

picodst.keys(filter_name="/.*mPMomentum[XYZ]/")
```
</details>

In [ ]:
px, py, pz = ???

<details>
    <summary><b>Hint for postponing the big download</b> (okay, maybe peek)</summary>

Remember that all of the array-fetching functions (`TTree.arrays`, `TBranch.array`, etc.) take `entry_start` and `entry_stop` as arguments.

You can quickly limit the number of baskets read by setting `entry_stop` to some small number, like 1000.

```python
picodst.arrays("Event.mEventId", entry_stop=1000)
```
<br>

When you've got something you like, you can then remove the `entry_stop=1000` to let it go through the whole file.
</details>

<details>
    <summary><b>Answer</b> (no peeking!)</summary>

There are several different ways to read the arrays, but the most straightforward are `TTree.arrays` and `TBranch.array`.

`TTree.arrays` can use the same `filter_name` that you identified with `TTree.keys`. But just calling it as-is returns a single array.

```python
single_array = picodst.arrays(filter_name="*mPMomentum[XYZ]")
```
<br>

**Solution 1:** You could, then, look at the `single_array.fields` ([ak.fields](https://awkward-array.readthedocs.io/en/latest/_auto/ak.fields.html)) to see what their names are and just pull them out.

```python
px = single_array["Track.mPMomentumX"]
py = single_array["Track.mPMomentumY"]
pz = single_array["Track.mPMomentumZ"]
```
<br>

**Solution 2:** Or you could pull them all out at once (without needing to know their names) with [ak.unzip](https://awkward-array.readthedocs.io/en/latest/_auto/ak.unzip.html).

```python
px, py, pz = ak.unzip(single_array)
```
<br>

**Solution 3:** Or, if you noticed the `how` parameter of `TTree.arrays`, you'd see that this gives you the desired form directly.

```python
px, py, pz = picodst.arrays(filter_name="*mPMomentum[XYZ]", how=tuple)
```
<br>

**Solution 4:** As one final alternative, you could get each branch object and use `TBranch.array` on each one individually. (For network traffic, this is three request-and-responses rather than one, so it's less efficient, but leave that for later performance tuning.)

```python
px = picodst["Track/Track.mPMomentumX"].array()
py = picodst["Track/Track.mPMomentumY"].array()
pz = picodst["Track/Track.mPMomentumZ"].array()
```
</details>

### Making an array of Lorentz vectors

Rather than calculating invariant mass from first principles, let's cast these as a `"Momentum4D"` with the Vector library.

In [28]:
import vector
vector.register_awkward()

**Exercise 2:** Combine the `px`, `py`, `pz`, and `M = 0.000511` (a constant) into an array of type

```
8004 * var * {"px": float32, "py": float32, "pz": float32, "M": float64}
```

It is _essential_ that the "`var`" is outside the record structure—that it is an array of lists of `px-py-pz-M` records, not an array of records of `px-py-pz-M` lists, such as

```
8004 * {"px": var * float32, "py": var * float32, "pz": var * float32, "M": float64}
```

_When that works,_ also add the name `"Momentum4D"` to those records, so that they become Lorentz vectors. The type should then look like

```
8004 * var * Momentum4D["px": float32, "py": float32, "pz": float32, "M": float64]
```

Both of these should compute instantaneously. If it's taking a long time, something's wrong.

**Hint:** The [ak.zip](https://awkward-array.readthedocs.io/en/latest/_auto/ak.zip.html) function is relevant. You might also use [ak.Array](https://awkward-array.readthedocs.io/en/latest/_auto/ak.Array.html) and/or [ak.with_name](https://awkward-array.readthedocs.io/en/latest/_auto/ak.with_name.html), and [ak.type](https://awkward-array.readthedocs.io/en/latest/_auto/ak.type.html) or just `.type` to look at the type.

In [29]:
pMom = ???

(<Array [[0, -0.142, 0, 0, 0, ... 0, 0, 0, 0]] type='8004 * var * float32'>,
 <Array [[0, 0.374, 0, 0, 0, ... 0, 0, 0, 0, 0]] type='8004 * var * float32'>,
 <Array [[0, 0.151, 0, 0, 0, ... 0, 0, 0, 0, 0]] type='8004 * var * float32'>)

<details>
    <summary><b>Answer</b> (no peeking!)</summary>

This is just what the `ak.zip` function does, when given a dict of field names to arrays. Don't set the `depth_limit`, as that would prevent their merger into an array of lists of records. The `"M"` field does not need to be an array—`ak.zip` will "broadcast" a constant into the same shape as the other fields.

```python
ak.zip({"px": px, "py": py, "pz": pz, "M": 0.000511})
```
<br>

To add the `"Momentum4D"` name to these fields, you could use `ak.with_name` explicitly:

```python
ak.with_name(ak.zip({"px": px, "py": py, "pz": pz, "M": 0.000511}), "Momentum4D")
```
<br>

or just add it as an argument in the original `ak.zip`:

```python
ak.zip({"px": px, "py": py, "pz": pz, "M": 0.000511}, with_name="Momentum4D")
```
<br>

The `ak.Array` constructor also has a `with_name` argument, so you could have used that, too.
</details>

### Aside: if you're averse to physical constants in your code

There are libraries for that. Particle is a "Pythonic PDG" and hepunits has unit conversions in the same units as Particle.

In [58]:
import particle, hepunits

particle.Particle.find("e-").mass / hepunits.GeV

0.0005109989460999999

Constant fields can be added to a list of records through assignment.

In [54]:
pMom["M"] = particle.Particle.find("e-").mass / hepunits.GeV

Now it's more precise.

In [57]:
pMom[0, 0, "M"]

0.0005109989460999999

### How many tracks are there per event? How many pairwise combinations?

The [ak.num](https://awkward-array.readthedocs.io/en/latest/_auto/ak.num.html) function returns the number of items in a `var` dimension, and NumPy's [np.mean](https://numpy.org/doc/stable/reference/generated/numpy.mean.html) computes an average.

In [60]:
ak.num(pMom)

<Array [2594, 2513, 2909, ... 2441, 2586, 2321] type='8004 * int64'>

In [65]:
np.mean(ak.num(pMom))

2611.1579210394802

**Exercise 3:** If we were to compute [ak.combinations](https://awkward-array.readthedocs.io/en/latest/_auto/ak.combinations.html) on these tracks (don't try it!), how many combinations would there be, on average?

Remember, for a list with $N$ items, `ak.combinations(dataset, 2)` produces "$N$ choose $2$" pairs, which is $\frac{N (N - 1)}{2}$.

<details>
    <summary><b>Answer</b> (no peeking!)</summary>

If $N$ were a single scalar, we would just compute the formula

```python
N = 10
(N * (N - 1)) / 2
```
<br>

In array-oriented computing, we can do the same thing with an array of values.

```python
tracks_in_each_event = ak.num(pMom)
pairs_in_each_event = (tracks_in_each_event * (tracks_in_each_event - 1)) / 2
np.mean(pairs_in_each_event)
```
<br>

which is 3.5 million pairs _in each event._
</details>

### Selecting electrons

Although we've created Lorentz vector objects of mass 0.000511 GeV, there's no guarantee that these tracks are electrons. In fact, many of them are not.

A good discriminating property of electrons is that they deposit most of their energy in an electromagnetic calorimeter. If the $p_T$ measured via track curvature is approximately equal to the energy of a matched calorimeter tower, then there's a better chance they're electrons.

In the C++ code ([star-picodst-W-reference-for-jim/macros/PicoDstAnalyzer.C](star-picodst-W-reference-for-jim/macros/PicoDstAnalyzer.C)), tracks are identified as having an electromagnetic shower match with

```c++
picoTrack->isBemcTrack()
```

and the matching shower is returned via

```c++
StPicoBEmcPidTraits *trait = dst->bemcPidTraits( picoTrack->bemcPidTraitsIndex() );
```

The The [star-picodst-W-reference-for-jim/StPicoTrack.h](star-picodst-W-reference-for-jim/StPicoTrack.h) defines these as

```c++
Bool_t isBemcTrack() const             { return (mBEmcPidTraitsIndex<0) ? false : true; }

Int_t bemcPidTraitsIndex() const       { return mBEmcPidTraitsIndex; }

static StPicoBEmcPidTraits* bemcPidTraits(Int_t i) { return (StPicoBEmcPidTraits*)picoArrays[StPicoArrays::BEmcPidTraits]->UncheckedAt(i); }
```

which we can find as branches in the TTree.

In [77]:
emc_index = picodst["Track.mBEmcPidTraitsIndex"].array()
emc_energy = picodst["EmcPidTraits.mBtowE"].array()

In [78]:
emc_index

<Array [[-1, -1, -1, -1, ... -1, -1, -1, -1]] type='8004 * var * int16'>

In [79]:
emc_energy

<Array [[0, 0, 1705, 32767, ... 0, 360, 0]] type='8004 * var * int16'>

**Exercise 4:** Make an array with the same shape as `pMom` containing _true_ if the corresponding track has an electromagnetic calorimeter match and _false_ if it does not.

It should have type ([ak.type](https://awkward-array.readthedocs.io/en/latest/_auto/ak.type.html))

```
8004 * var * bool
```

In [ ]:
isBemcTrack = ???

It should then be possible to compute

In [86]:
pMom_isemc = pMom[isBemcTrack]
emc_index_isemc = emc_index[isBemcTrack]

and the average numbers of _selected_ tracks is much fewer.

In [88]:
np.mean(ak.num(pMom_isemc))

16.81984007996002

In [89]:
np.mean(ak.num(emc_index_isemc))

16.81984007996002

<details>
    <summary><b>Answer</b> (no peeking!)</summary>

Comparison operators, `<` and `>`, apply array-at-a-time, like anything else.

```python
isBemcTrack = (emc_index >= 0)
```
<br>

Be sure to get the direction of the comparison right: in the C++ code, `mBEmcPidTraitsIndex<0` means `isBemcTrack` is _false_.
</details>